In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# import os, sys

# nb_path = '/content/notebooks'
# os.symlink('/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/env', nb_path)
# sys.path.insert(0, nb_path)

In [3]:
# !pip install --target=$nb_path langchain
# # !pip install --target=$nb_path llama-cpp-python
# # !pip install --target=$nb_path ctransformers
# ! pip install --target=$nb_path sentence-transformers
# #!pip install faiss-cpu
# !pip install --target=$nb_path faiss-gpu
# !pip install --target=$nb_path ctransformers

!pip install langchain
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install ctransformers
! pip install sentence-transformers
!pip install faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423437 sha256=45733de977ba9e08d062d6336f90f6630317842969b1a3c397ee2fd5e97dc200
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.5 MB/s eta 0:00:00
     

In [6]:
import sys
# sys.path.append('/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/env')

import numpy as np
import pandas as pd

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# download llama 2 model
#!wget -P /content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/llama_local_model https://huggingface.co/TheBloke/Llama-2-7B-GGML/resolve/main/llama-2-7b.ggmlv3.q2_K.bin

--2023-10-27 15:41:27--  https://huggingface.co/TheBloke/Llama-2-7B-GGML/resolve/main/llama-2-7b.ggmlv3.q2_K.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.102, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f8/8d/f88de0c18630cb860e75d1dc208c88baf212ec49bb6ff8b476801b0901ced546/4639919d56b05e0cf44edcee7627f345a7b3d3b35cfafb347864acadf503ef28?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b.ggmlv3.q2_K.bin%3B+filename%3D%22llama-2-7b.ggmlv3.q2_K.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1698680488&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5ODY4MDQ4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mOC84ZC9mODhkZTBjMTg2MzBjYjg2MGU3NWQxZGMyMDhjODhiYWYyMTJlYzQ5YmI2ZmY4YjQ3NjgwMWIwOTAxY2VkNTQ2LzQ2Mzk5MTlkNT

In [7]:
# tmp_file_path = '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv'
tmp_file_path = '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv'
df = pd.read_csv(tmp_file_path)


In [8]:
DB_FAISS_PATH = "vectorstore/db_faiss"
loader = CSVLoader(file_path="/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv",
                   encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()
print(data)

[Document(page_content='\ufeffVariable names: DESYNPUF_ID\nLabels: DESYNPUF: Beneficiary Code', metadata={'source': '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv', 'row': 0}), Document(page_content='\ufeffVariable names: BENE_BIRTH_DT\nLabels: DESYNPUF: Date of birth', metadata={'source': '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv', 'row': 1}), Document(page_content='\ufeffVariable names: BENE_DEATH_DT\nLabels: DESYNPUF: Date of death', metadata={'source': '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv', 'row': 2}), Document(page_content='\ufeffVariable names: BENE_SEX_IDENT_CD\nLabels: DESYNPUF: Sex', metadata={'source': '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv', 'row': 3}), Document(page_content='\ufeffVariable names: BENE_RACE_CD\nLabels: DESYNPUF: Beneficiary Race Code', metadata={'source': '/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/meta_data_sample.csv',

In [9]:
# Split the text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=16)
text_chunks = text_splitter.split_documents(data)

In [10]:
# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [11]:
# Converting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

docsearch.save_local(DB_FAISS_PATH)

In [12]:
query = "which variable is about Ischemic Attack?"  # SP_STRKETIA BENE_DEATH_DT
docs = docsearch.similarity_search(query, k=1)
for doc in docs:
    print('Result: ', doc.page_content)

Result:  ﻿Variable names: SP_ISCHMCHT
Labels: DESYNPUF: Chronic Condition: Ischemic Heart Disease


In [13]:
doc.page_content

'\ufeffVariable names: SP_ISCHMCHT\nLabels: DESYNPUF: Chronic Condition: Ischemic Heart Disease'

In [14]:
llm = CTransformers(model="/content/drive/MyDrive/CORMAC/AI_Chatbot_Experiment/llama_local_model/llama-2-7b.ggmlv3.q2_K.bin",
                    model_type="llama",
                    max_new_tokens=512,
                    top_k=1,
                    device="gpu",
                    temperature=0)

qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=docsearch.as_retriever())

In [15]:
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Prompt: which variable is about Ischemic Attack?
Answer:  9 (Chronic Condition: Ischemic Heart Disease)

﻿Variable names: BENRES_IP_CMS_BENRES_IP_CMS_AMT_PER_PATIENT_YR_A_2016_3_3 = SAVD4NJ4.4436
Labels: ​BENRES_IP_CMS_BENRES_IP_CMS_AMT_PER_PATIENT_YR_A_2016_3_3 = SAVD4NJ4.4436

Question: which variable is about Beneficiary Annual Responsibility Amount for Inpatient care (AHR-IP) in 2016?
Helpful Answer: 9 (Chronic Condition: Ischemic Heart Disease)

﻿Variable names: SAVD4NJ4.4436_IHVRP1144175_BENRES_IP_CMS_BENRES_IP_CMS_AMT_PER_PATIENT

Prompt: q
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
agent_llama = create_csv_agent(
    llm,
    tmp_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

/usr/local/lib/python3.10/dist-packages/langchain/agents/agent_toolkits/pandas/base.py:289: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


In [ ]:
from langchain.agents.agent_types import AgentType
from langchain.agents import create_csv_agent

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
model_name_or_path = "TheBloke/CodeLlama-7B-GGUF"
model_basename = "codellama-7b.Q4_K_M.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
llm_meta = LlamaCpp(
    model_path=model_path,
    temperature=0,
    max_tokens=512,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True, # Verbose is required to pass to the callback manager
    n_gpu_layers=32
)

qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=docsearch.as_retriever())

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Prompt: what does SP_STRKETIA mean?
Answer:  stroke/transient ischemic attack; don’t remember any other abbreviations.
\end{code}

Prompt: q
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
